<a href="https://colab.research.google.com/github/Yoonsun222/CapstoneDesign/blob/main/%EC%BA%A1%EC%8A%A4%ED%86%A4%EB%94%94%EC%9E%90%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df1 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/미먼1.csv", encoding = 'euc-kr', parse_dates=['등록일자'])

df2 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/미먼2.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df3 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/미먼3.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df4 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/미먼4.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df5  = pd.read_csv("/content/drive/MyDrive/캡디테스트파일/MAPO_LIB_AIR_2022.03.07-03.13.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df6  = pd.read_csv("/content/drive/MyDrive/캡디테스트파일/MAPO_LIB_AIR_2022.03.14-03.20 (1).csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df7  = pd.read_csv("/content/drive/MyDrive/캡디테스트파일/MAPO_LIB_AIR_2022.03.21-03.27 (1).csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df8 = pd.read_csv("/content/drive/MyDrive/캡디테스트파일/MAPO_LIB_AIR_2022.03.28-04.03 (1).csv", encoding = 'euc-kr', parse_dates=['등록일자'])

df = pd.concat([df5,df6,df7,df8,df1,df2,df3,df4])

df['등록일자_str'] = df["등록일자"].dt.strftime("%Y-%m-%d")
df['시간'] = df["등록일자"].dt.strftime("%H").astype(int)+1
df =df[df.columns[[0,1,2,3,4,5,6,8,9]]]

df

,시리얼,온도(℃),습도(%),미세먼지(㎍/㎥),휘발성유기화합물(ppb),초미세먼지(㎍/㎥),등록일자,등록일자_str,시간
0,V01G1613537,1210,36,40,235,29,2022-03-07 08:57:00,2022-03-07,9
1,V01G1613542,1205,18,76,255,56,2022-03-07 08:57:00,2022-03-07,9
2,V01G1613601,1144,26,18,139,13,2022-03-07 08:57:00,2022-03-07,9
3,V01G1613603,1142,27,47,441,34,2022-03-07 08:57:00,2022-03-07,9
4,V01G1613606,1191,20,18,155,13,2022-03-07 08:58:00,2022-03-07,9
...,...,...,...,...,...,...,...,...,...
1747,V01G1613629,1254,25,23,577,17,2022-05-01 21:55:00,2022-05-01,22
1748,V01G1613630,1213,27,15,1180,11,2022-05-01 21:55:00,2022-05-01,22
1749,V01G1613632,1241,29,22,1285,16,2022-05-01 21:55:00,2022-05-01,22
1750,V01G1613633,1234,28,56,1204,41,2022-05-01 21:55:00,2022-05-01,22


,시리얼,온도(℃),습도(%),미세먼지(㎍/㎥),휘발성유기화합물(ppb),초미세먼지(㎍/㎥),등록일자,등록일자_str,시간
0,V01G1613537,1210,36,40,235,29,2022-03-07 08:57:00,2022-03-07,9
1,V01G1613542,1205,18,76,255,56,2022-03-07 08:57:00,2022-03-07,9
2,V01G1613601,1144,26,18,139,13,2022-03-07 08:57:00,2022-03-07,9
3,V01G1613603,1142,27,47,441,34,2022-03-07 08:57:00,2022-03-07,9
4,V01G1613606,1191,20,18,155,13,2022-03-07 08:58:00,2022-03-07,9
...,...,...,...,...,...,...,...,...,...
1747,V01G1613629,1254,25,23,577,17,2022-05-01 21:55:00,2022-05-01,22
1748,V01G1613630,1213,27,15,1180,11,2022-05-01 21:55:00,2022-05-01,22
1749,V01G1613632,1241,29,22,1285,16,2022-05-01 21:55:00,2022-05-01,22
1750,V01G1613633,1234,28,56,1204,41,2022-05-01 21:55:00,2022-05-01,22


In [ ]:



dust_df1 = pd.read_csv("/content/drive/MyDrive/캡디테스트파일/OBS_부유분진_TIM_20220510004304.csv",encoding = 'euc-kr', parse_dates=['등록일자'] )
dust_df2 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/OBS_부유분진_TIM_20220509183223.csv",encoding = 'euc-kr', parse_dates=['등록일자'] )
dust_df = pd.concat([dust_df1,dust_df2])

dust_df['등록일자_str'] = dust_df["등록일자"].dt.strftime("%Y-%m-%d")
dust_df['시간'] = dust_df["등록일자"].dt.strftime("%H").astype(int)
dust_df

,등록일자,1시간평균 미세먼지농도(㎍/㎥),등록일자_str,시간
0,2022-03-07 01:00:00,35,2022-03-07,1
1,2022-03-07 02:00:00,33,2022-03-07,2
2,2022-03-07 03:00:00,38,2022-03-07,3
3,2022-03-07 04:00:00,38,2022-03-07,4
4,2022-03-07 05:00:00,32,2022-03-07,5
...,...,...,...,...
608,2022-04-30 20:00:00,32,2022-04-30,20
609,2022-04-30 21:00:00,41,2022-04-30,21
610,2022-04-30 22:00:00,30,2022-04-30,22
611,2022-04-30 23:00:00,44,2022-04-30,23


In [ ]:
weather_df1 = pd.read_csv("/content/drive/MyDrive/캡디테스트파일/OBS_ASOS_TIM_20220510004610.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
weather_df2 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/기상청데이터.csv", encoding = 'euc-kr', parse_dates=['등록일자'])

weather_df = pd.concat([weather_df1,weather_df2])

weather_df['등록일자_str'] = weather_df["등록일자"].dt.strftime("%Y-%m-%d")
weather_df['시간'] = weather_df["등록일자"].dt.strftime("%H").astype(int)



weather_df


,등록일자,기온(°C),풍속(m/s),습도(%),전운량(10분위),등록일자_str,시간
0,2022-03-07 01:00:00,-0.9,1.4,56,7,2022-03-07,1
1,2022-03-07 02:00:00,-1.4,1.3,54,6,2022-03-07,2
2,2022-03-07 03:00:00,-1.9,1.0,58,5,2022-03-07,3
3,2022-03-07 04:00:00,-2.0,1.8,58,5,2022-03-07,4
4,2022-03-07 05:00:00,-2.3,1.6,56,6,2022-03-07,5
...,...,...,...,...,...,...,...
643,2022-04-30 20:00:00,12.0,2.6,79,8,2022-04-30,20
644,2022-04-30 21:00:00,11.7,2.4,83,8,2022-04-30,21
645,2022-04-30 22:00:00,11.4,3.3,86,7,2022-04-30,22
646,2022-04-30 23:00:00,11.3,3.1,86,6,2022-04-30,23


In [ ]:
data = pd.merge(left = df , right = weather_df, how = "inner", on = ["등록일자_str","시간"])
data = pd.merge(left = data , right = dust_df, how = "inner", on = ["등록일자_str","시간"])
data.head(100)


data = data.replace(-9999,np.nan)
data = data.replace(-999,np.nan)
data = data.dropna(axis=0)
data = data.drop_duplicates(['시리얼','등록일자'], keep='first')
data = data.sort_values(by=['시리얼','등록일자'] ,ascending=True)


data

,시리얼,온도(℃),습도(%)_x,미세먼지(㎍/㎥),휘발성유기화합물(ppb),초미세먼지(㎍/㎥),등록일자_x,등록일자_str,시간,등록일자_y,기온(°C),풍속(m/s),습도(%)_y,전운량(10분위),등록일자,1시간평균 미세먼지농도(㎍/㎥)
0,V01G1613537,1210,36,40,235,29,2022-03-07 08:57:00,2022-03-07,9,2022-03-07 09:00:00,0.4,2.1,58,8,2022-03-07 09:00:00,41
16,V01G1613537,1210,36,40,235,29,2022-03-07 09:57:00,2022-03-07,10,2022-03-07 10:00:00,3.0,0.7,51,2,2022-03-07 10:00:00,54
77,V01G1613537,1210,36,40,235,29,2022-03-07 14:57:00,2022-03-07,15,2022-03-07 15:00:00,8.4,2.9,29,7,2022-03-07 15:00:00,43
160,V01G1613537,1210,36,40,235,29,2022-03-08 10:57:00,2022-03-08,11,2022-03-08 11:00:00,8.0,1.5,50,4,2022-03-08 11:00:00,52
228,V01G1613537,1210,36,40,235,29,2022-03-08 12:57:00,2022-03-08,13,2022-03-08 13:00:00,10.8,2.9,46,7,2022-03-08 13:00:00,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20890,V01G1613637,1228,17,39,348,28,2022-04-27 15:56:00,2022-04-27,16,2022-04-27 16:00:00,23.4,2.8,23,0,2022-04-27 16:00:00,217
20924,V01G1613637,1228,17,39,348,28,2022-04-27 16:56:00,2022-04-27,17,2022-04-27 17:00:00,22.8,3.9,26,0,2022-04-27 17:00:00,190
20980,V01G1613637,1228,17,39,348,28,2022-04-27 18:56:00,2022-04-27,19,2022-04-27 19:00:00,19.6,3.6,31,0,2022-04-27 19:00:00,170
21002,V01G1613637,1228,17,39,348,28,2022-04-27 19:56:00,2022-04-27,20,2022-04-27 20:00:00,18.0,2.1,33,0,2022-04-27 20:00:00,144


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data["시리얼"] = encoder.fit_transform(data["시리얼"])

data = data.reset_index()
data


,index,시리얼,온도(℃),습도(%)_x,미세먼지(㎍/㎥),휘발성유기화합물(ppb),초미세먼지(㎍/㎥),등록일자_x,등록일자_str,시간,등록일자_y,기온(°C),풍속(m/s),습도(%)_y,전운량(10분위),등록일자,1시간평균 미세먼지농도(㎍/㎥)
0,0,0,1210,36,40,235,29,2022-03-07 08:57:00,2022-03-07,9,2022-03-07 09:00:00,0.4,2.1,58,8,2022-03-07 09:00:00,41
1,16,0,1210,36,40,235,29,2022-03-07 09:57:00,2022-03-07,10,2022-03-07 10:00:00,3.0,0.7,51,2,2022-03-07 10:00:00,54
2,77,0,1210,36,40,235,29,2022-03-07 14:57:00,2022-03-07,15,2022-03-07 15:00:00,8.4,2.9,29,7,2022-03-07 15:00:00,43
3,160,0,1210,36,40,235,29,2022-03-08 10:57:00,2022-03-08,11,2022-03-08 11:00:00,8.0,1.5,50,4,2022-03-08 11:00:00,52
4,228,0,1210,36,40,235,29,2022-03-08 12:57:00,2022-03-08,13,2022-03-08 13:00:00,10.8,2.9,46,7,2022-03-08 13:00:00,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21026,20890,35,1228,17,39,348,28,2022-04-27 15:56:00,2022-04-27,16,2022-04-27 16:00:00,23.4,2.8,23,0,2022-04-27 16:00:00,217
21027,20924,35,1228,17,39,348,28,2022-04-27 16:56:00,2022-04-27,17,2022-04-27 17:00:00,22.8,3.9,26,0,2022-04-27 17:00:00,190
21028,20980,35,1228,17,39,348,28,2022-04-27 18:56:00,2022-04-27,19,2022-04-27 19:00:00,19.6,3.6,31,0,2022-04-27 19:00:00,170
21029,21002,35,1228,17,39,348,28,2022-04-27 19:56:00,2022-04-27,20,2022-04-27 20:00:00,18.0,2.1,33,0,2022-04-27 20:00:00,144


In [ ]:
total_data = data[data.columns[[1,2,3,4,5,6,9,10,11,12,13,14,16]]]
total_data = total_data.replace(-9999,np.nan)
total_data = total_data.replace(-999,np.nan)
total_data = total_data.dropna(axis=0)
total_data.head(4)


total_data.columns = ['Serial','arduino_temp', 'arduino_hum','arduino_pm25','arduino_VOC','arduino_pm10','time','datetime','TMP','WSD','REH','SKY','pm25Value']
total_data

,Serial,arduino_temp,arduino_hum,arduino_pm25,arduino_VOC,arduino_pm10,time,datetime,TMP,WSD,REH,SKY,pm25Value
0,0,1210,36,40,235,29,9,2022-03-07 09:00:00,0.4,2.1,58,8,41
1,0,1210,36,40,235,29,10,2022-03-07 10:00:00,3.0,0.7,51,2,54
2,0,1210,36,40,235,29,15,2022-03-07 15:00:00,8.4,2.9,29,7,43
3,0,1210,36,40,235,29,11,2022-03-08 11:00:00,8.0,1.5,50,4,52
4,0,1210,36,40,235,29,13,2022-03-08 13:00:00,10.8,2.9,46,7,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21026,35,1228,17,39,348,28,16,2022-04-27 16:00:00,23.4,2.8,23,0,217
21027,35,1228,17,39,348,28,17,2022-04-27 17:00:00,22.8,3.9,26,0,190
21028,35,1228,17,39,348,28,19,2022-04-27 19:00:00,19.6,3.6,31,0,170
21029,35,1228,17,39,348,28,20,2022-04-27 20:00:00,18.0,2.1,33,0,144


In [ ]:
total_data.to_csv('/content/drive/MyDrive/pre_capstone2.csv',index=False)


In [ ]:
total_data = pd.read_csv("/content/drive/MyDrive/pre_capstone2 (2).csv", encoding = 'euc-kr')

In [ ]:
 total_data['arduino_temp'] = (total_data['arduino_temp']/10)-100

In [ ]:
condition = []

for idx, i in enumerate(total_data['datetime']):
  date, time = i.split(" ")
  if time != 9:
    condition.append(idx)


total_data['datetime'] = total_data.loc[condition, 'datetime']

total_data = total_data.dropna(axis=0)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

total_data = pd.read_csv("/content/drive/MyDrive/캡디테스트파일/캡디_Realdata.csv", encoding = 'euc-kr') 

In [ ]:
conditionlist = [
  (total_data['arduino_pm25.1'] <= 10) ,
  (total_data['arduino_pm25.1'] <= 30) & (total_data['arduino_pm25.1'] >= 10),
  (total_data['arduino_pm25.1'] >= 30)]
choicelist = [0, 1, 2]
total_data['arduino_pm25.1_class'] = np.select(conditionlist, choicelist, default='Not Specified')

total_data

,날짜,arduino_temp,arduino_hum,arduino_pm25,arduino_pm10,TMP,WSD,REH,SKY,pm25Value,arduino_pm25.1,arduino_pm25.1_class
0,2022-05-17 10:00,24,53,11,12,19.3,3.8,56,0,26,10,0
1,2022-05-17 10:30,24,52,9,10,19.3,3.8,56,0,26,11,1
2,2022-05-17 11:00,24,56,11,12,20.8,2.9,53,0,25,9,0
3,2022-05-17 14:00,23,49,16,16,22.3,3.0,52,0,25,15,1
4,2022-05-17 14:30,23,48,18,18,22.3,3.0,52,0,25,16,1
5,2022-05-17 15:00,22,52,15,15,21.5,3.9,53,0,27,18,1
6,2022-05-17 15:30,22,51,17,18,21.5,3.9,53,0,27,15,1
7,2022-05-17 16:00,22,52,15,16,21.3,2.8,55,0,21,17,1
8,2022-05-17 16:30,23,52,17,18,21.3,2.8,55,0,21,15,1
9,2022-05-17 17:00,23,52,20,19,20.2,3.3,59,0,20,17,1


In [ ]:
corr_df= total_data.corr(method='pearson')

corr_df[corr_df.columns[-1]]

arduino_temp     -0.309646
arduino_hum       0.656640
arduino_pm25      0.980530
arduino_pm10      0.978602
TMP               0.326951
WSD              -0.304245
REH               0.439429
SKY              -0.401995
pm25Value         0.603755
arduino_pm25.1    1.000000
Name: arduino_pm25.1, dtype: float64

In [ ]:
X =  total_data[total_data.columns[[1,2,3,4,5,6,7,8,9]]]
Y = total_data[total_data.columns[[-1]]]
X

,arduino_temp,arduino_hum,arduino_pm25,arduino_pm10,TMP,WSD,REH,SKY,pm25Value
0,24,53,11,12,19.3,3.8,56,0,26
1,24,52,9,10,19.3,3.8,56,0,26
2,24,56,11,12,20.8,2.9,53,0,25
3,23,49,16,16,22.3,3.0,52,0,25
4,23,48,18,18,22.3,3.0,52,0,25
5,22,52,15,15,21.5,3.9,53,0,27
6,22,51,17,18,21.5,3.9,53,0,27
7,22,52,15,16,21.3,2.8,55,0,21
8,23,52,17,18,21.3,2.8,55,0,21
9,23,52,20,19,20.2,3.3,59,0,20


In [ ]:
Y[]

,arduino_pm25.1_class
0,1
1,1
2,0
3,0
4,0
...,...
19709,0
19710,1
19711,0
19712,1


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=222)




scaler = MinMaxScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_train = pd.DataFrame(X_scaled_train, columns=X_train.columns, index=X_train.index)

X_scaled_test = scaler.fit_transform(X_test)
X_scaled_test = pd.DataFrame(X_scaled_test, columns=X_test.columns, index=X_test.index)




In [ ]:
# X_scaled_train = np.reshape(X_scaled_train, (2055210, 10 , 2))

In [ ]:
# hist = model.fit(X_train_scaled, Y_train, epochs=10, batch_size=1, validation_split=0.2)

In [ ]:
from sklearn.neural_network import MLPClassifier
model=MLPClassifier(hidden_layer_sizes=55, activation='tanh', solver='adam')
model.fit(X_scaled_train, y_train)
pred_train=model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)
pred_test=model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


0.9333333333333333

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(X_scaled_train, y_train)
pred_train=model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)
pred_test=model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.9333333333333333

In [ ]:
import joblib
joblib.dump(model, '/content/drive/MyDrive/Real_model.pkl')

['/content/drive/MyDrive/Real_model.pkl']

In [ ]:
import joblib

classifer = joblib.load('/content/drive/MyDrive/ml_class_model4.pkl')
new_observation = [[25,58,20,25,23.5,2.3,47,0,33]]

# 샘플의 클래스를 예측합니다.
classifer.predict(new_observation)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  "X does not have valid feature names, but"


array(['1'], dtype='<U1')

In [ ]:
import firebase_admin
from firebase_admin import db
from firebase_admin import credentials

if not firebase_admin._apps:
    cred = credentials.Certificate('/content/drive/MyDrive/캡스톤디자인/capstone-9ecef-firebase-adminsdk-1k7jj-61adc62b10.json')
    firebase_admin.initialize_app(cred, {
      'databaseURL': 'https://capstone-9ecef-default-rtdb.firebaseio.com/'
    })
  
db.reference(u'Logs').update({'speed' : 2})

In [ ]:
model.score(X_scaled_test, y_test)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
model=KNeighborsRegressor()
model.fit(X_scaled_train, y_train)
pred_train=model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)
pred_test=model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

-0.0017863862555349552

In [ ]:


import numpy as np
from sklearn.metrics import mean_squared_error 
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈련   데이터 RMSE:", np.sqrt(MSE_train))
print("테스트 데이터 RMSE:", np.sqrt(MSE_test))


In [ ]:
from scipy.stats import randint
param_distribs={'hidden_layer_sizes': randint(low=10, high=100), 'solver': ['sgd', 'adam'], 
             'activation': ['tanh', 'relu']}
from sklearn.model_selection import RandomizedSearchCV
random_search=RandomizedSearchCV(MLPClassifier(), 
                                 param_distributions=param_distribs, n_iter=10, cv=5)
random_search.fit(X_scaled_train, y_train)

print("Best Parameter: {}".format(random_search.best_params_))
print("Best Score: {:.4f}".format(random_search.best_score_))
print("TestSet Score: {:.4f}".format(random_search.score(X_scaled_test, y_test)))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/li

Best Parameter: {'activation': 'relu', 'hidden_layer_sizes': 88, 'solver': 'adam'}
Best Score: 0.8611
TestSet Score: 0.9333


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [ ]:
from keras.layers import *
from keras.models import *
from keras.utils import *


model = Sequential()
model.add(LSTM(256, input_dim=1, input_length=10, stateful=True))
model.add(LSTM(64, return_sequences=True))
model.add(Dense(1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

